In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer


In [2]:
doc="Nice and friendly place with excellent food and friendly and helpful staff. You need a car though. The children wants to go back! Playground and animals entertained them and they felt like at home. I also recommend the dinner! Great value for the price!"
doc1="okay experience. fine"
doc2="terrible experience. bad stuff"


In [3]:
# tokenization and POS tagging
def filteration(text):
    sentences = nltk.sent_tokenize(text)
    tokens = [nltk.word_tokenize(sent) for sent in sentences]
    taggedlist=[]
    for tok in tokens:        
         taggedlist.append(nltk.pos_tag(tok))
    return taggedlist

In [4]:
wnl = nltk.WordNetLemmatizer()

In [5]:
#for words with POS tags NN, JJ, V, R their avg polarity score is calculated as summation(positive_score)- summation(negative_score)
#polarity of a sentence is evaluated by adding polarities of all the words in the sentence.
#polarity of a review is brought out by adding polarities of all the sentences in the review.
def polarity(taggedlist):
    score_list=[]
    for idx1,taggedsent in enumerate(taggedlist):
        score_list.append([])
        for idx2,t in enumerate(taggedsent):
                newtag=''
                lemmatized=wnl.lemmatize(t[0])
                if t[1].startswith('NN'):
                    newtag='n'
                elif t[1].startswith('JJ'):
                    newtag='a'
                elif t[1].startswith('V'):
                    newtag='v'
                elif t[1].startswith('R'):
                    newtag='r'
                else:
                    newtag=''       
                if(newtag!=''):    
                    synsets = list(swn.senti_synsets(lemmatized, newtag))
                    #print synsets
               
                    score=0
                    if(len(synsets)>0):
                        for syn in synsets:
                            score+=syn.pos_score()-syn.neg_score()
                        score_list[idx1].append(score/len(synsets))
            

    #print score_list

    sentence_sentiment=[]
    for score_sent in score_list:
        if (len(score_sent)>0):
            sentence_sentiment.append(sum([word_score for word_score in score_sent])/len(score_sent))
        else: continue
    #print sentence_sentiment
    total=0;
    for i in sentence_sentiment:
        total+=i;
    return total

In [6]:
#positive sentiment score gives a positive review, negative sentiment score gives negative review 
def sentiment(total):
    if(total> 0):
        rev="positive"
        r=1
    elif(total<0):
        rev="negative"
        r=0
    return r

In [7]:
# dataset includes amazon sports product reviews
data=pd.read_json('data.json', lines=True)
data.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1881509818,"[0, 0]",5,This came in on time and I am veru happy with ...,"01 26, 2014",AIXZKN4ACSKI,David Briner,Woks very good,1390694400
1,1881509818,"[1, 1]",5,I had a factory Glock tool that I was using fo...,"02 2, 2012",A1L5P841VIO02V,Jason A. Kramer,Works as well as the factory tool,1328140800
2,1881509818,"[2, 2]",4,If you don't have a 3/32 punch or would like t...,"02 28, 2012",AB2W04NI4OEAD,J. Fernald,"It's a punch, that's all.",1330387200
3,1881509818,"[0, 0]",4,This works no better than any 3/32 punch you w...,"02 5, 2012",A148SVSWKTJKU6,"Jusitn A. Watts ""Maverick9614""",It's a punch with a Glock logo.,1328400000
4,1881509818,"[0, 0]",4,I purchased this thinking maybe I need a speci...,"04 23, 2013",AAAWJ6LW9WMOO,Material Man,"Ok,tool does what a regular punch does.",1366675200


In [8]:
sentiment(polarity(filteration(doc2)))

0

In [9]:
sentiment(polarity(filteration(doc1)))

1

In [10]:
sentiment(polarity(filteration(doc)))

1

In [11]:
data=pd.read_json('data.json', lines=True)
data_fil=data[:100]["reviewText"]


In [12]:
data_fil.head()

0    This came in on time and I am veru happy with ...
1    I had a factory Glock tool that I was using fo...
2    If you don't have a 3/32 punch or would like t...
3    This works no better than any 3/32 punch you w...
4    I purchased this thinking maybe I need a speci...
Name: reviewText, dtype: object

In [13]:
a=[]
for dat in data_fil:
    a.append(sentiment(polarity(filteration(dat))))

In [14]:
review=data["reviewText"]

In [15]:
rating= data["overall"]

In [16]:
reviewer_name= data["reviewerName"]

In [17]:
rev_rat=pd.DataFrame(columns=["review","rating","reviewerName"])

In [18]:
rev_rat["review"]=review[:100]

In [19]:
rev_rat["rating"]=rating[:100]

In [20]:
rev_rat["sentiment"]=a
rev_rat["reviewerName"]=reviewer_name[:100]


In [21]:
rev_rat.head()

,review,rating,reviewerName,sentiment
0,This came in on time and I am veru happy with ...,5,David Briner,1
1,I had a factory Glock tool that I was using fo...,5,Jason A. Kramer,1
2,If you don't have a 3/32 punch or would like t...,4,J. Fernald,1
3,This works no better than any 3/32 punch you w...,4,"Jusitn A. Watts ""Maverick9614""",1
4,I purchased this thinking maybe I need a speci...,4,Material Man,0


In [22]:
rev_rat.head(5)


,review,rating,reviewerName,sentiment
0,This came in on time and I am veru happy with ...,5,David Briner,1
1,I had a factory Glock tool that I was using fo...,5,Jason A. Kramer,1
2,If you don't have a 3/32 punch or would like t...,4,J. Fernald,1
3,This works no better than any 3/32 punch you w...,4,"Jusitn A. Watts ""Maverick9614""",1
4,I purchased this thinking maybe I need a speci...,4,Material Man,0


In [23]:
#taking into account reviews with rating above 3 are positive and rest are negative
rev_rat['expected']=[1 if x>3 else 0 for x in rev_rat.rating]

In [24]:
rev_rat.head()

,review,rating,reviewerName,sentiment,expected
0,This came in on time and I am veru happy with ...,5,David Briner,1,1
1,I had a factory Glock tool that I was using fo...,5,Jason A. Kramer,1,1
2,If you don't have a 3/32 punch or would like t...,4,J. Fernald,1,1
3,This works no better than any 3/32 punch you w...,4,"Jusitn A. Watts ""Maverick9614""",1,1
4,I purchased this thinking maybe I need a speci...,4,Material Man,0,1


In [25]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [26]:
accuracy_score(rev_rat["sentiment"], rev_rat["expected"])

0.85

In [27]:
confusion_matrix(rev_rat["sentiment"], rev_rat["expected"])

array([[ 3,  8],
       [ 7, 82]])

In [28]:
f1_score(rev_rat["sentiment"], rev_rat["expected"])

0.9162011173184358